In [1]:
!pip install git+https://github.com/shumingma/transformers.git

  Cloning https://github.com/shumingma/transformers.git to /tmp/pip-req-build-ni9141rt
  Running command git clone --filter=blob:none --quiet https://github.com/shumingma/transformers.git /tmp/pip-req-build-ni9141rt
  Resolved https://github.com/shumingma/transformers.git to commit 4a01efe84d0120dc2545ff2de445082400d87407
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11413940 sha256=e75f665c920fa5cbd6be585064713eda82b92900695cb449df077da4c5c0865c
  Stored in directory: /tmp/pip-ephem-wheel-cache-dn474lb3/wheels/2c/2a/7c/3be0c30fb51a7becc4bcbb536739ae9ed9cc7e633ffbfaf63b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [ ]:
import os
from huggingface_hub import login
login(token=os.environ.get("HF_TOKEN"))

In [2]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_name)
model

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [3]:
import torch
from torch import nn
from transformers import PreTrainedModel, PretrainedConfig
class Qwen25Classifier(PreTrainedModel):
  def __init__(self, config: PretrainedConfig, base_model: nn.Module, num_classes: int):
    super().__init__(config)
    self.num_labels = num_classes
    self.config = config

    self.qwen_base = base_model

    self.classifier = nn.Linear(in_features=config.hidden_size, out_features=num_classes)

  def forward(self, input_ids, attention_mask) -> torch.Tensor:
    base_outputs = self.qwen_base(
        input_ids=input_ids,
        attention_mask=attention_mask,
    )

    last_hidden_state = base_outputs.last_hidden_state

    sequence_lengths = torch.max(attention_mask.sum(dim=1) - 1,
                                 torch.zeros(1, dtype=torch.long, device=input_ids.device))
    pooled_output = last_hidden_state[torch.arange(last_hidden_state.shape[0],
                                      device=last_hidden_state.device),
                                      sequence_lengths]

    return self.classifier(pooled_output)

In [1]:
from transformers import AutoConfig
model_name = 'Qwen/Qwen2.5-1.5B'
config = AutoConfig.from_pretrained(model_name)
config

Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "max_position_embeddings": 131072,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": 131072,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.0.dev0",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 151936
}

In [2]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, config=config)
model

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2Model(
  (embed_tokens): Embedding(151936, 1536)
  (layers): ModuleList(
    (0-27): 28 x Qwen2DecoderLayer(
      (self_attn): Qwen2Attention(
        (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
        (k_proj): Linear(in_features=1536, out_features=256, bias=True)
        (v_proj): Linear(in_features=1536, out_features=256, bias=True)
        (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
        (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
        (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((1536,), eps=1e-06)
  (rotary_emb): Qwen2RotaryEmbedding()
)

In [3]:
dummy_input = "Hello, how're you doing "
inputs = tokenizer(dummy_input, return_tensors="pt")
inputs

{'input_ids': tensor([[9707,   11, 1246, 2299,  498, 3730,  220]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [4]:
dummy_ouput = model(**inputs)
dummy_ouput

BaseModelOutputWithPast(last_hidden_state=tensor([[[-0.5243, -1.8065, -0.0066,  ...,  0.5989, -1.5053,  1.4487],
         [-0.1421, -2.4847, -0.4080,  ...,  2.2751, -0.5769,  1.4527],
         [ 0.7948,  1.2146, -2.5325,  ...,  3.8421,  0.1352,  1.2220],
         ...,
         [-0.9298,  3.3723,  0.1516,  ...,  3.7858,  1.6857, -0.0501],
         [-0.4974,  0.7981,  1.4112,  ...,  2.1235,  1.4826, -1.0152],
         [-3.7456,  1.0669,  4.7619,  ...,  3.7346,  0.7447, -3.6289]]],
       grad_fn=<MulBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x78f5a7b091d0>, hidden_states=None, attentions=None)

In [10]:
dummy_ouput.last_hidden_state[:,-1,:]

tensor([[-3.7456,  1.0669,  4.7619,  ...,  3.7346,  0.7447, -3.6289]],
       grad_fn=<SliceBackward0>)